In [1]:
# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# import the libraries mentioned above
from apeer_ometiff_library import io, processing, omexmlClass
import czifile as zis
import xmltodict
import os
import numpy as np
import ipywidgets as widgets
import napari
import imgfileutils as imf
import xml.etree.ElementTree as ET

In [4]:
filename = r'testdata/CellDivision_T=10_Z=15_CH=2_DCV_small.czi'

# To read the CZI metadata one can use the czifile functionality

# get czi object using the filename
czi = zis.CziFile(filename)

# get the metadata and close the czi file object
mdczi = czi.metadata()
czi.close()

# parse the XML into a dictionary
metadatadict_czi = xmltodict.parse(mdczi)

In [6]:
# get the array and the metadata
array, metadata, add_metadata = imf.get_array_czi(filename, return_addmd=False)
print('Array Shape: ', array.shape)

Array Shape:  (1, 10, 2, 15, 256, 256)


In [7]:
import time

# switch to qt5 backend for napari viewer and wait a few seconds
%gui qt5
time.sleep(5)

In [8]:
# configure napari viewer manually - check array shape and dimensions order carefully 
    
# get the scalefactors
scalefactors = imf.get_scalefactor(metadata)
print(scalefactors)

viewer = napari.Viewer()

# add every channel as a single layer
for ch in range(metadata['SizeC']):
    chname = metadata['Channels'][ch]
    channel = array[:, :, ch, :, :, :]
    clim = [channel.min(), np.round(channel.max() * 0.85)]
    # add channel as a layer
    viewer.add_image(channel,
                     name=chname,
                     scale=(1, 1, 1, scalefactors['zx'], 1, 1),
                     contrast_limits=clim,
                     blending='additive',
                     gamma=0.85)

{'xy': 1.0, 'zx': 3.516}


jupyter nbconvert MIAP_Read_and_Display_Images.ipynb --to slides --post serve